In [100]:
from neo4j import GraphDatabase, basic_auth

import pprint

In [101]:
# 1) Змоделювати предметну область онлайн-магазину:

delete_query = [
    "MATCH (i:Item) DETACH DELETE i;",
    "MATCH (c:Customer) DETACH DELETE c;",
    "MATCH (o:Order) DETACH DELETE o;"
]

create_query = """
CREATE 
(i1:Item {item_id: 1, category: "Phone", model: "Google Pixel 5", producer: "Google", price: 699, color: "Just Black"}),
(i2:Item {item_id: 2, category: "Laptop", model: "MacBook Air", producer: "Apple", price: 999, color: "Silver"}),
(i3:Item {item_id: 3, category: "Tablet", model: "iPad Pro", producer: "Apple", price: 799, color: "Space Gray"}),
(i4:Item {item_id: 4, category: "Smartwatch", model: "Apple Watch Series 6", producer: "Apple", price: 399, color: "Space Black"}),
(i5:Item {item_id: 5, category: "Headphones", model: "AirPods Pro", producer: "Apple", price: 249, color: "White"}),
(i6:Item {item_id: 6, category: "Smartphone", model: "Samsung Galaxy S21", producer: "Samsung", price: 799, color: "Phantom Gray"}),
(i7:Item {item_id: 7, category: "Smart TV", model: "LG OLED55C1PUB", producer: "LG", price: 1499, color: "Black"}),
(i8:Item {item_id: 8, category: "Gaming Console", model: "PlayStation 5", producer: "Sony", price: 499, color: "White"}),
(i9:Item {item_id: 9, category: "Camera", model: "Canon EOS R5", producer: "Canon", price: 3899, color: "Black"}),

(c1:Customer {customer_id: 1, name: "Olga", surname: "Ivanova", phones: [123456789, 987654321], address: "123 Main St, Anytown, USA"}),
(c2:Customer {customer_id: 2, name: "Pavel", surname: "Sidorov", phones: [555555555, 777777777], address: "456 Oak St, Othertown, USA"}),
(c3:Customer {customer_id: 3, name: "Maria", surname: "Kuznetsova", phones: [333333333, 666666666], address: "789 Elm St, Thistown, USA"}),
(c4:Customer {customer_id: 4, name: "Dmitry", surname: "Popov", phones: [888888888, 999999999], address: "101 Pine St, Anothertown, USA"}),
(c5:Customer {customer_id: 5, name: "Elena", surname: "Volkova", phones: [111111111, 222222222], address: "135 Cedar St, Lasttown, USA"}),
(c6:Customer {customer_id: 6, name: "Ivan", surname: "Petrov", phones: [444444444], address: "246 Birch St, Nexttown, USA"}),

(o1:Order {order_id: 201514, date: "2015-04-15"}),
(o2:Order {order_id: 201515, date: "2015-04-16"}),
(o3:Order {order_id: 201516, date: "2015-04-17"}),
(o4:Order {order_id: 201517, date: "2015-04-18"}),
(o5:Order {order_id: 201518, date: "2015-04-19"})
"""

bought_query = """
MATCH (c:Customer {customer_id: $customer_id}), (o:Order {order_id: $order_id})
MERGE (c)-[:BOUGHT]->(o)
"""

contains_query = """
MATCH (o:Order {order_id: $order_id}), (i:Item {item_id: $item_id})
MERGE (o)-[:CONTAINS]->(i)
"""

view_query = """
MATCH (c:Customer {customer_id: $customer_id}), (i:Item {item_id: $item_id})
MERGE (c)-[:VIEW]->(i)
"""

update_total_sum_query = """
MATCH (o:Order {order_id: $order_id})
WITH o, [(o)-[:CONTAINS]->(i:Item) | i.price] AS prices
WITH o, reduce(total = 0, price IN prices | total + price) AS new_total_sum
SET o.total_sum = new_total_sum;
"""

In [102]:
driver = GraphDatabase.driver(
  "neo4j://3.238.2.126:7687",
  auth=basic_auth("neo4j", "dresses-jaw-branch"))


def create_data():
    with driver.session() as session:
        for query in delete_query:
            session.run(query)
        session.run(create_query)  


def add_contains_relationship(order_id, item_id):
    with driver.session() as session:
        session.run(contains_query, order_id=order_id, item_id=item_id)
        session.run(update_total_sum_query, order_id=order_id)
        

def add_bought_relationship(customer_id, order_id):
    with driver.session() as session:
        session.run(bought_query, customer_id=customer_id, order_id=order_id)


def add_view_relationship(customer_id, item_id):
    with driver.session() as session:
        session.run(view_query, customer_id=customer_id, item_id=item_id)


create_data()

add_contains_relationship(201514, 1)
add_contains_relationship(201514, 2)
add_contains_relationship(201515, 3)
add_contains_relationship(201516, 5)
add_contains_relationship(201516, 7)
add_contains_relationship(201516, 2)
add_contains_relationship(201517, 1)
add_contains_relationship(201518, 2)
add_contains_relationship(201518, 4)

add_bought_relationship(1, 201514)
add_bought_relationship(2, 201515)
add_bought_relationship(2, 201514)
add_bought_relationship(3, 201518)
add_bought_relationship(4, 201516)
add_bought_relationship(4, 201517)
add_bought_relationship(5, 201516)

add_view_relationship(1, 2)
add_view_relationship(1, 7)
add_view_relationship(3, 5)
add_view_relationship(3, 1)
add_view_relationship(3, 2)
add_view_relationship(6, 9)

In [103]:
# 2) Знайти Items які входять в конкретний Order (за Order id) 

def find_items_in_order(order_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (o:Order {order_id: $order_id})-[:CONTAINS]->(i:Item)
            RETURN i
            """,
            order_id=order_id
        )
        items = [record["i"]._properties for record in result]
        return items


order_id = 201514
items_in_order = find_items_in_order(order_id)
pprint.pprint(items_in_order)

[{'category': 'Laptop',
  'color': 'Silver',
  'item_id': 2,
  'model': 'MacBook Air',
  'price': 999,
  'producer': 'Apple'},
 {'category': 'Phone',
  'color': 'Just Black',
  'item_id': 1,
  'model': 'Google Pixel 5',
  'price': 699,
  'producer': 'Google'}]


In [104]:
# 3) Підрахувати вартість конкретного Order
 
def get_order_total(order_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (o:Order {order_id: $order_id})
            RETURN o.total_sum AS total_cost
            """,
            order_id=order_id
        )
        total_cost = result.single()["total_cost"]
        return total_cost


order_id = 201514
total_cost = get_order_total(order_id)
print(f"Total cost of order {order_id}: ${total_cost}")

Total cost of order 201514: $1698


In [109]:
# 4) Знайти всі Orders конкретного Customer

def find_orders_by_customer(customer_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer {customer_id: $customer_id})-[:BOUGHT]->(o:Order)
            RETURN o
            """,
            customer_id=customer_id
        )
        orders = [record["o"]._properties for record in result]
        return orders


customer_id = 4
orders = find_orders_by_customer(customer_id)
pprint.pprint(orders)

[{'date': '2015-04-18', 'order_id': 201517, 'total_sum': 699},
 {'date': '2015-04-17', 'order_id': 201516, 'total_sum': 2747}]


In [115]:
# 5) Знайти всі Items куплені конкретним Customer (через його Orders)

def find_items_by_customer(customer_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer {customer_id: $customer_id})-[:BOUGHT]->(:Order)-[:CONTAINS]->(i:Item)
            RETURN i
            """,
            customer_id=customer_id
        )
        items = [record["i"]._properties for record in result]
        return items


customer_id = 4
items = find_items_by_customer(customer_id)
pprint.pprint(items)

[{'category': 'Phone',
  'color': 'Just Black',
  'item_id': 1,
  'model': 'Google Pixel 5',
  'price': 699,
  'producer': 'Google'},
 {'category': 'Laptop',
  'color': 'Silver',
  'item_id': 2,
  'model': 'MacBook Air',
  'price': 999,
  'producer': 'Apple'},
 {'category': 'Smart TV',
  'color': 'Black',
  'item_id': 7,
  'model': 'LG OLED55C1PUB',
  'price': 1499,
  'producer': 'LG'},
 {'category': 'Headphones',
  'color': 'White',
  'item_id': 5,
  'model': 'AirPods Pro',
  'price': 249,
  'producer': 'Apple'}]


In [116]:
# 6) Знайти загальну кількість Items куплені конкретним Customer (через його Order)

def count_items_bought_by_customer(customer_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer {customer_id: $customer_id})-[:BOUGHT]->(:Order)-[:CONTAINS]->(i:Item)
            RETURN COUNT(i) AS total_items_bought
            """,
            customer_id=customer_id
        )
        total_items_bought = result.single()["total_items_bought"]
        return total_items_bought


customer_id = 4
total_items_bought = count_items_bought_by_customer(customer_id)
print("Total items bought by customer:", total_items_bought)

Total items bought by customer: 4


In [117]:
# 7) Знайти для Customer на яку загальну суму він придбав товарів (через його Order)

def total_spent_by_customer(customer_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer {customer_id: $customer_id})-[:BOUGHT]->(o:Order)
            RETURN SUM(o.total_sum) AS total_spent_by_customer
            """,
            customer_id=customer_id
        )
        total_spent_by_customer = result.single()["total_spent_by_customer"]
        return total_spent_by_customer


customer_id = 4
total_spent = total_spent_by_customer(customer_id)
print("Total spent by customer:", total_spent)

Total spent by customer: 3446


In [129]:
# 8) Знайти скільки разів кожен товар був придбаний, відсортувати за цим значенням

def count_purchases_per_item():
    with driver.session() as session:
        result = session.run(
            """
            MATCH (i:Item)<-[:CONTAINS]-(:Order)
            RETURN i, COUNT(*) AS purchases
            ORDER BY purchases DESC
            """
        )
        purchases_per_item = [(record["i"], record["purchases"]) for record in result]
        return purchases_per_item


purchases_per_item = count_purchases_per_item()
for item, purchases in purchases_per_item:
    print(f"Item {item['item_id']} was purchased {purchases} times")

Item 2 was purchased 3 times
Item 1 was purchased 2 times
Item 3 was purchased 1 times
Item 4 was purchased 1 times
Item 5 was purchased 1 times
Item 7 was purchased 1 times


In [141]:
# 9) Знайти всі Items переглянуті (view) конкретним Customer

def find_viewed_items(customer_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer {customer_id: $customer_id})-[:VIEW]->(i:Item)
            RETURN i
            """,
            customer_id=customer_id
        )
        viewed_items = [record["i"]._properties for record in result]
        return viewed_items

customer_id = 1
viewed_items = find_viewed_items(customer_id)
pprint.pprint(viewed_items)

[{'category': 'Smart TV',
  'color': 'Black',
  'item_id': 7,
  'model': 'LG OLED55C1PUB',
  'price': 1499,
  'producer': 'LG'},
 {'category': 'Laptop',
  'color': 'Silver',
  'item_id': 2,
  'model': 'MacBook Air',
  'price': 999,
  'producer': 'Apple'}]


In [140]:
# 10) Знайти інші Items що купувались разом з конкретним Item (тобто всі Items що входять до Order-s разом з даними Item)
def find_related_items(item_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (i1:Item {item_id: $item_id})<-[:CONTAINS]-(o:Order)-[:CONTAINS]->(i2:Item)
            WHERE i1 <> i2
            RETURN DISTINCT i2
            """,
            item_id=item_id
        )
        items = [record["i2"]._properties for record in result]
        return items

item_id = 5
related_items = find_related_items(item_id)
pprint.pprint(related_items)

[{'category': 'Laptop',
  'color': 'Silver',
  'item_id': 2,
  'model': 'MacBook Air',
  'price': 999,
  'producer': 'Apple'},
 {'category': 'Smart TV',
  'color': 'Black',
  'item_id': 7,
  'model': 'LG OLED55C1PUB',
  'price': 1499,
  'producer': 'LG'}]


In [134]:
# 11) Знайти Customers які купили даний конкретний Item

def find_customers_who_bought_item(item_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer)-[:BOUGHT]->(:Order)-[:CONTAINS]->(i:Item)
            WHERE i.item_id = $item_id
            RETURN DISTINCT c
            """,
            item_id=item_id
        )
        customers = [record["c"]._properties for record in result]
        return customers

item_id = 1
customers = find_customers_who_bought_item(item_id)
pprint.pprint(customers)

[{'address': '101 Pine St, Anothertown, USA',
  'customer_id': 4,
  'name': 'Dmitry',
  'phones': [888888888, 999999999],
  'surname': 'Popov'},
 {'address': '456 Oak St, Othertown, USA',
  'customer_id': 2,
  'name': 'Pavel',
  'phones': [555555555, 777777777],
  'surname': 'Sidorov'},
 {'address': '123 Main St, Anytown, USA',
  'customer_id': 1,
  'name': 'Olga',
  'phones': [123456789, 987654321],
  'surname': 'Ivanova'}]


In [136]:
# 12) Знайти для певного Customer(а) товари, які він переглядав, але не купив

def find_items_viewed_but_not_bought(customer_id):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (c:Customer {customer_id: $customer_id})-[:VIEW]->(i:Item)
            WHERE NOT EXISTS {
                (c)-[:BOUGHT]->(:Order)-[:CONTAINS]->(i)
            }
            RETURN i
            """,
            customer_id=customer_id
        )
        items = [record["i"]._properties for record in result]
        return items

customer_id = 1
items = find_items_viewed_but_not_bought(customer_id)
pprint.pprint(items)

[{'category': 'Smart TV',
  'color': 'Black',
  'item_id': 7,
  'model': 'LG OLED55C1PUB',
  'price': 1499,
  'producer': 'LG'}]
